**This notebook belongs to Engin Deniz ALPMAN.**

In [1]:
import torch
import torch.nn as nn
from PIL import Image
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import os

In [3]:
import math

In [4]:
PATH= Path('data/mnist/')

In [5]:
Path.ls = lambda x: list(x.iterdir())

In [6]:
class Dataset():
    def __init__(self, x, y): 
        self.x,self.y = x,y
    def __len__(self): 
        return len(self.x)
    def __getitem__(self, i): 
        return self.x[i],self.y[i]

In [7]:
class DataLoader():
    def __init__(self, ds, bs): 
        self.ds, self.bs = ds, bs
    def __iter__(self):
        n = len(self.ds)
        l = torch.randperm(n)

        
        for i in range(0, n, self.bs): 
            idxs_l = l[i:i+self.bs]
            yield self.ds[idxs_l]

In [8]:
def init(size):
    return torch.randn(size, dtype=torch.float32).requires_grad_()

In [9]:
def linear_layer(xb):
    return xb @ w + b

In [30]:
def accuracy(preds, actuals):
    preds = preds.sigmoid()
    return ((preds > 0.5) == actuals).float().mean()

In [11]:
def validation_acc(model):
    return torch.stack([accuracy(model(xb), yb) for xb, yb in valid_dl]).mean().item()

In [31]:
def loss_func(preds, targets):
    preds = preds.sigmoid()
    return torch.where(targets==1, 1-preds, preds).mean()

In [13]:
# params is list of variables that our model will update
def train(model, params, lr, epochs=5, valid_epoch=5):
    for epoch in range(epochs):
        for xb, yb in train_dl:
            preds = model(xb)
            loss = loss_func(preds, yb)
            loss.backward()
            for p in params:
                p.data = p.data - p.grad * lr
                p.grad.zero_()
        
        if epoch % valid_epoch == 0:
            print(validation_acc(model))
        

In [14]:
train_3 = torch.stack([torch.tensor(np.array(Image.open(img)), dtype = torch.float32) for img in (PATH/"train/3").ls()]) / 255

In [15]:
train_2 = torch.stack([torch.tensor(np.array(Image.open(img)), dtype = torch.float32) for img in (PATH/"train/2").ls()]) / 255

In [16]:
valid_3 = torch.stack([torch.tensor(np.array(Image.open(img)), dtype = torch.float32) for img in (PATH/"validation/3").ls()]) / 255

In [17]:
valid_2 = torch.stack([torch.tensor(np.array(Image.open(img)), dtype = torch.float32) for img in (PATH/"validation/2").ls()]) / 255

In [18]:
train_x = torch.cat([train_3, train_2]).view(-1, 28*28)

In [19]:
train_y = torch.tensor([1] * len(train_3) + [0] * len(train_2))

In [20]:
valid_x = torch.cat([valid_3, valid_2]).view(-1, 28*28)

In [21]:
valid_y = torch.tensor([1] * len(valid_3) + [0] * len(valid_2))

In [22]:
train_y = train_y.unsqueeze(1)

In [23]:
valid_y = valid_y.unsqueeze(1)

In [24]:
ds_train = Dataset(train_x, train_y)

In [25]:
ds_valid = Dataset(valid_x, valid_y)

In [26]:
train_dl = DataLoader(ds_train, bs = 512)

In [27]:
valid_dl = DataLoader(ds_valid, bs = 512)

In [28]:
w = init((28*28,1))
b = init(1)
lr = 1
params = w, b

In [32]:
train(linear_layer, params, lr=1, epochs= 30, valid_epoch=2)

0.960156261920929
0.9515625238418579
0.960156261920929
0.9644531011581421
0.959765613079071
0.9664062261581421
0.962890625
0.965624988079071
0.9644531011581421
0.962109386920929
0.9671875238418579
0.9613281488418579
0.961718738079071
0.962890625
0.9683593511581421
